## Datan analysointi K2020_2 - Tehtävät kerta 7

Palauta Optiman palautuslaatikkoon viimeistään su 12.4. Kirjoita palautuslaatikon kommenttikenttään, mitkä tehtävät olet tehnyt kokonaan/osittain.

Opiskelijan nimi:

# 1. Asunnon hinta
Tiedostossa https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta7/teht1.txt on (kuvitteellisia) asuntojen myyntihintoja ja niiden tietoja (pinta-ala, makuuhuoneiden lukumäärä ja saunan olemassaolo).

Tee **lineaarien regressiomalli**, jossa selittävinä muuttujina on pinta-ala, makuuhuoneiden lukumäärä ja saunan olemassaolo ja selitettävänä muuttujana myyntihinta. 

(Joudut muuttamaan sauna-kentän numeeriseksi)

Laske mallille selityskerroin.

Kokeile sitten mitä saat selityskertoimiksi, jos käytät selittävinä muuttuujina vain kahta kolmesta mahdollisesta.

Piirrä parhaasta mallista kaavio ennustetuista ja havaituista arvoista.

Tässä ei jaeta dataa opetus- ja testidataan.


## 2. Ostaako

Tiedostossa https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta7/teht2.txt on dataa henkilön tuloista, sukupuolesta ja siviilisäädystä. Lisäksi on tieto onko henkilö ostanut tuotteen vai ei.

Tee **päätöspuu** asetuksella max_depth=4, joka yrittää selittää ostamista muiden kenttien avulla. Testaa sitten mallia ja  tulosta `score` ja `confusion_matrix`.

Joudut muuttamaan joitain kenttiä numeerisiksi.


Tee päätöspuusta vuokaavio (dot-tiedosto), visualisoi se ja päättele manuaalisesti sen avulla miten puu luokittelisi seuraavat tiedot:
* tulot 60000, naimisissa, nainen
* tulot 30000, ei-naimisissa, mies

Kirjoita vastaukseen kommenttina päättelyketjut.

Tässä ei jaeta dataa opetus- ja testidataan.

## 3. Jyväskylän kämpät
Tiedostossa https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta7/teht5.txt on tietoja Jyväskylän toteutuneista kerrostaloasuntokaupoista  12 kk ajalta.

Jaa data opetusdataan (60%) ja testidataan (40%) ja tee opetusdatalla **lineaarinen regressiomalli** hinnan ennustamiseksi kentistä:
* m2
* rakennusvuosi
* hissi (muunna 0/1)
* kunto (muunna numeeriseksi)
* tee uusi kenttä onkoKeskustassa, joka saa arvon 1, jos kaupunginosa = Keskusta

Tsekkaa onko datassa puuttuvia arvoja ja korvaa ne ko. kentän mediaanilla. 

Testaa saatua mallia testidatalla.  
Piirrä hajontakaavioon ennusteen antamat arvot vs toteutuneet arvot.

## 4. Koripallo

Tiedostossa https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta7/nba_logreg.csv on 1340 NBA-pelaajan tilastoja heidän tulokaskaudeltaan. 

[kenttien selitys (eng)](https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta7/nba.png)

Viimeinen sarake kertoo onko pelaaja sittemmin pelannut NBA:ssa yli 5 kautta. Kokeile kuinka hyvin **logistinen regressio** pystyy ennustamaan tämän muiden kentttien (pl. pelaajan nimi) avulla. Korvaa tarvittaessa puuttuvat arvot ko. kentän mediaanilla. Tulosta score ja confusion matrix.


Tee sitten sama kokeilu **päätöspuun** avulla. Tulosta tästä myös kuinka paljon eri kentät vaikuttavat mallin mukaan ennustettavaan muuttujaan.

Tässä ei tarvitse jakaa aineistoa opetus- ja testidataan.

## 5. Sienet
Tiedostossa https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta7/mushrooms.csv on tietoja sienistä (kenttien selitykset [englanniksi](https://student.labranet.jamk.fi/~varpe/datananal2k2020/kerta7/mushrooms.txt)).

Pyri ennustamaan kentän _class_ arvoa muiden kenttien avulla **päätöspuulla**. 
Kentät ovat luokkamuotoisia, tyyliin 
>gill-attachment: attached=a,descending=d,free=f,notched=n

Tee siis ensin (selittävistä) kentistä dummy-kentät pandasin get_dummies-funktiolla, voit tehdä sen kerralla kaikille sarakkeille.

Jaa data opetusdataan ja testidataan 60/40%. 

Kokeile päätöspuulle max_depth-arvoja 2, 3, 4 ja 5 ja printtaa niiden antamat scoret ja confusion_matrixit.